- Install the additional package requirements for this bonus notebook by uncommenting and running the following cell:

In [ ]:
# pip install -r requirements-extra.txt

# Using BytePair encodding from `tiktoken`

In [2]:
import importlib.metadata

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.5.1


In [3]:
import tiktoken

tik_tokenizer = tiktoken.get_encoding("gpt2")

text = "Hello, world. Is this-- a test?"

In [4]:
integers = tik_tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 995, 13, 1148, 428, 438, 257, 1332, 30]


In [5]:
strings = tik_tokenizer.decode(integers)

print(strings)

Hello, world. Is this-- a test?


In [6]:
print(tik_tokenizer.n_vocab)

50257


# Using the original Byte-pair encoding implementation used in GPT-2

In [7]:
from bpe_openai_gpt2 import get_encoder, download_vocab

In [8]:
download_vocab()

Fetching encoder.json: 1.04Mit [00:00, 3.03Mit/s]                                                   
Fetching vocab.bpe: 457kit [00:00, 2.36Mit/s]                                                       


In [9]:
orig_tokenizer = get_encoder(model_name="gpt2_model", models_dir=".")

In [10]:
integers = orig_tokenizer.encode(text)

print(integers)

[15496, 11, 995, 13, 1148, 428, 438, 257, 1332, 30]


In [11]:
strings = orig_tokenizer.decode(integers)

print(strings)

Hello, world. Is this-- a test?


# Using the BytePair Tokenizer in Hugging Face transformers

In [13]:
import transformers

transformers.__version__

/Users/sebastian/miniforge3/envs/book/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'4.33.2'

In [14]:
from transformers import GPT2Tokenizer

hf_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [15]:
hf_tokenizer(strings)["input_ids"]

[15496, 11, 995, 13, 1148, 428, 438, 257, 1332, 30]

# A quick performance benchmark

In [16]:
with open('../01_main-chapter-code/the-verdict.txt', 'r', encoding='utf-8') as f:
    raw_text = f.read()

In [17]:
%timeit orig_tokenizer.encode(raw_text)

4.12 ms ± 41.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [18]:
%timeit tik_tokenizer.encode(raw_text)

1.75 ms ± 8.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [19]:
%timeit hf_tokenizer(raw_text)["input_ids"]

Token indices sequence length is longer than the specified maximum sequence length for this model (5145 > 1024). Running this sequence through the model will result in indexing errors


9.12 ms ± 856 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [20]:
%timeit hf_tokenizer(raw_text, max_length=5145, truncation=True)["input_ids"]

8.63 ms ± 247 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
